# Radiomic Feature Post-processing 

In [ ]:
# Intention Hassan transform: Render feature independent of gray level discretization.

In [1]:
import sys
sys.path.append('./../utils/')

import os
import re

import numpy as np
import pandas as pd

import ioutil
import transforms

from processing import PostProcessor
from sklearn.preprocessing import StandardScaler

In [2]:
# Load Hassan transformation functions.
hassan_transf = transforms.hassan_transforms(None)

In [3]:
# Globals
THRESH = 1e-8
FILTER_TYPE = 'sigma'

In [4]:
true_index = np.load('./../../../data_source/patient_id.npy')

## Raw Features

In [5]:
# Individual file names from feature extraction.
ct_feature_files = [
    #'./../../../data_source/radiomic_features/gauss/ct_32_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/ct_64_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/ct_128_05gauss_config.csv',
    './../../../data_source/radiomic_features/sqroot/ct_32_sqroot_config.csv',
    './../../../data_source/radiomic_features/sqroot/ct_64_sqroot_config.csv',
    './../../../data_source/radiomic_features/sqroot/ct_128_sqroot_config.csv'
]
pet_feature_files = [
    #'./../../../data_source/radiomic_features/gauss/pet_32_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/pet_64_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/pet_128_05gauss_config.csv',
    './../../../data_source/radiomic_features/sqroot/pet_32_sqroot_config.csv',
    './../../../data_source/radiomic_features/sqroot/pet_64_sqroot_config.csv',
    './../../../data_source/radiomic_features/sqroot/pet_128_sqroot_config.csv'
]

In [6]:
ct_processor = PostProcessor(
    path_to_features=ct_feature_files, 
    indices=true_index, 
    filter_type='squareroot', 
    error_dir='./../../../data_source/to_analysis/sqroot/errors/'
)
ct_features = ct_processor.produce(drop_redundant=True, drop_missing=True)

Num missing features: 1
Num missing features: 1
Num missing features: 1


In [7]:
pet_processor = PostProcessor(
    path_to_features=pet_feature_files, 
    indices=true_index, 
    filter_type='squareroot', 
    error_dir='./../../../data_source/to_analysis/sqroot/errors/'
)
pet_features = pet_processor.produce(drop_redundant=True, drop_missing=True)

In [8]:
# Check number of extracted features.
for ct_dset, pet_dset in zip(ct_features, pet_features):
    print(np.shape(ct_dset), np.shape(pet_dset))

(198, 91) (198, 92)
(198, 91) (198, 92)
(198, 91) (198, 92)


In [ ]:
def get_discr_level(path_to_file):
    return re.findall('\d+', path_to_file)[0]


# Save data to disk.
for num, (ct_set, pet_set) in enumerate(zip(ct_data, pet_data)):
    # Update column labels.
    ct_set.columns = [
        'CT{}_{}'.format(get_discr_level(ct_labels[num]), label) 
        for label in ct_set.columns
    ]
    pet_set.columns = [
        'PET{}_{}'.format(get_discr_level(pet_labels[num]), label) 
        for label in pet_set.columns
    ]

In [ ]:
shape_feats = pd.read_csv('./../../../data_source/to_analysis/radiomics_shape.csv', index_col=0)
clinical = pd.read_csv('./../../../data_source/to_analysis/clinical_params.csv', index_col=0)
pet_params = pd.read_csv('./../../../data_source/to_analysis/pet_params.csv', index_col=0)

In [ ]:
X = pd.concat(
    (clinical, shape_feats, pet_params, ct_set, pet_set), 
    axis=1
)
X.shape

In [ ]:
X.head()

In [ ]:
X.to_csv('./../../../data_source/to_analysis/gauss05_complete.csv')

## Combine Feature Sets

In [ ]:
ct_concat = pd.concat(ct_data, axis=1)
pet_concat = pd.concat(pet_data, axis=1)

ct_concat.shape, pet_concat.shape

In [ ]:
# Save to disk.
#ct_concat.to_csv(path_ct_concat)
#pet_concat.to_csv(path_pet_concat)

In [ ]:

pet_params.head()

In [ ]:
ct_concat.head()

In [ ]:
pet_concat.head()

In [ ]:
path_ct_concat = './../../../data_source/to_analysis/ct_no_filter_concat.csv'
path_pet_concat = './../../../data_source/to_analysis/pet_no_filter_concat.csv'

In [ ]:
X = pd.concat(
    (clinical, ct_set, pet_set, pet_params), axis=1
)

In [ ]:
X.head()

In [ ]:
X.to_csv('./../../../data_source/to_analysis/squareroot_complete.csv')

In [ ]:
ct_set.to_csv('./../../../data_source/to_analysis/sqroot/ct_128_sqroot.csv')

In [ ]:
pet_set.to_csv('./../../../data_source/to_analysis/sqroot/pet_16_sqroot.csv')

In [ ]:
shape_feats = pd.read_csv('./../../../data_source/to_analysis/radiomics_shape.csv', index_col=0)